## Train a character-level GPT on some text data

The inputs here are simple text files, which we chop up to individual characters and then train GPT on. So you could say this is a char-transformer instead of a char-rnn. Doesn't quite roll off the tongue as well. In this example we will feed it some Shakespeare, which we'll get it to predict character-level.

In [1]:
# set up logging
import logging
logging.basicConfig(
        format="%(asctime)s - %(levelname)s - %(name)s -   %(message)s",
        datefmt="%m/%d/%Y %H:%M:%S",
        level=logging.INFO,
)

In [2]:
# make deterministic
from mingpt.utils import set_seed
set_seed(42)

In [3]:
import numpy as np
import torch
import torch.nn as nn
from torch.nn import functional as F

In [4]:
import math
from torch.utils.data import Dataset

class CharDataset(Dataset):

    def __init__(self, data, block_size):
        chars = sorted(list(set(data)))
        data_size, vocab_size = len(data), len(chars)
        print('data has %d characters, %d unique.' % (data_size, vocab_size))
        
        self.stoi = { ch:i for i,ch in enumerate(chars) }
        self.itos = { i:ch for i,ch in enumerate(chars) }
        self.block_size = block_size
        self.vocab_size = vocab_size
        self.data = data
    
    def __len__(self):
        return len(self.data) - self.block_size

    def __getitem__(self, idx):
        # grab a chunk of (block_size + 1) characters from the data
        chunk = self.data[idx:idx + self.block_size + 1]
        # encode every character to an integer
        dix = [self.stoi[s] for s in chunk]
        """
        arrange data and targets so that the first i elements of x
        will be asked to predict the i-th element of y. Notice that
        the eventual language model will actually make block_size
        individual predictions at the same time based on this data,
        so we are being clever and amortizing the cost of the forward
        pass of the network. So for example if block_size is 4, then
        we could e.g. sample a chunk of text "hello", the integers in
        x will correspond to "hell" and in y will be "ello". This will
        then actually "multitask" 4 separate examples at the same time
        in the language model:
        - given just "h", please predict "e" as next
        - given "he" please predict "l" next
        - given "hel" predict "l" next
        - given "hell" predict "o" next
        
        In addition, because the DataLoader will create batches of examples,
        every forward/backward pass during traning will simultaneously train
        a LOT of predictions, amortizing a lot of computation. In particular,
        for a batched input of integers X (B, T) where B is batch size These probabilities are computed using the softmax function on the logits (raw scores).and
        T is block_size and Y (B, T), the network will during training be
        simultaneously training to make B*T predictions, all at once! Of course,
        at test time we can paralellize across batch B, but unlike during training
        we cannot parallelize across the time dimension T - we have to run
        a forward pass of the network to recover the next single character of the 
        sequence along each batch dimension, and repeatedly always feed in a next
        character to get the next one.
        
        So yes there is a big asymmetry between train/test time of autoregressive
        models. During training we can go B*T at a time with every forward pass,
        but during test time we can only go B at a time, T times, with T forward 
        passes.
        """
        x = torch.tensor(dix[:-1], dtype=torch.long)
        y = torch.tensor(dix[1:], dtype=torch.long)
        return x, y


In [5]:
block_size = 128 # spatial extent of the model for its context

In [6]:
# you can download this file at https://github.com/karpathy/char-rnn/blob/master/data/tinyshakespeare/input.txt
text = open('input.txt', 'r').read() # don't worry we won't run out of file handles
train_dataset = CharDataset(text, block_size) # one line of poem is roughly 50 characters

data has 1115394 characters, 65 unique.


In [7]:
from mingpt.model import GPT, GPTConfig
mconf = GPTConfig(train_dataset.vocab_size, train_dataset.block_size,
                  n_layer=8, n_head=8, n_embd=512)
model = GPT(mconf)

03/07/2025 17:57:27 - INFO - mingpt.model -   number of parameters: 2.535219e+07


In [ ]:
from mingpt.trainer import Trainer, TrainerConfig

# initialize a trainer instance and kick off training
torch.cuda.set_per_process_memory_fraction(0.7, device=0)  # Use 70% of available memory
tconf = TrainerConfig(max_epochs=2, batch_size=64, learning_rate=6e-4,
                      lr_decay=True, warmup_tokens=512*20, final_tokens=2*len(train_dataset)*block_size,
                      num_workers=4)
trainer = Trainer(model, train_dataset, None, tconf)
trainer.train()

  0%|          | 0/17427 [00:00<?, ?it/s]


OutOfMemoryError: CUDA out of memory. Tried to allocate 16.00 MiB. GPU 0 has a total capacity of 11.89 GiB of which 2.85 GiB is free. Process 16778 has 32.89 MiB memory in use. Including non-PyTorch memory, this process has 8.48 GiB memory in use. 8.33 GiB allowed; Of the allocated memory 8.31 GiB is allocated by PyTorch, and 9.48 MiB is reserved by PyTorch but unallocated. If reserved but unallocated memory is large try setting PYTORCH_CUDA_ALLOC_CONF=expandable_segments:True to avoid fragmentation.  See documentation for Memory Management  (https://pytorch.org/docs/stable/notes/cuda.html#environment-variables)

In [15]:
# alright, let's sample some character-level Shakespeare
from mingpt.utils import sample

context = "O God, O God!"
x = torch.tensor([train_dataset.stoi[s] for s in context], dtype=torch.long)[None,...].to(trainer.device)
y = sample(model, x, 2000, temperature=1.0, sample=True, top_k=10)[0]
completion = ''.join([train_dataset.itos[int(i)] for i in y])
print(completion)

O God, O God!XiPXDiti:P.yy.:iiiXiXzzixXUzy'.Xyy'XXIDzB:Bi'3UiG:'iyz:fh'BXUzXU33UD:iihii:ByB3:BzBz'yx3yzhy':zii:UXXUXUXBW:B:fUzXyUzRL:ffRx!RfW:yXf!ynXvByRLh:B,!RBX!xBXX,Bc!f:!B::fn:J:y!XB!cXXSncWD,DXBj:fn,!RR!nRDWX!B,qRLRc:RnWWS!BRRWWW,c!g:R$cccg!WSWSScRWcPWcjWRqnW:j,juh!WRcujRD,hP!RnqchhhnoWRhhSzQXP!XcXhGncno!h!hoqR-ujQchP,SRgcgjwQPhPjqcnQnqqvqwqhhSQXhXWqySFPX!SSqRFAhQSvSAmyknv-R-AhnRPXqnFcXcvncXSR-XXMPRRMMwPXXF&XFWq-cnRPsnqRRWW-nXWq-wPXWnwcgw-qw&fPWnnuWWP-fqZuRPXRXPufWunoPWnfWWhjjhPn;HfRhjfWRGWffRfhwh&fXgERBHRfnLj-XfB;BJhgRBxqPSSBJfgZZqwqfhPZZqqnPvEjn-hnnZPPRgPng;LnPfP-gPXqRgZqq,-SqRqR!RnP!Pm,mnjPj,gRXnRjRXSW&nR&,SnP,;L;,;LngS&;WWvnLnHRnssRJRP,S,PREPSDPLEV&;xgRDnSxPEEgXxBgDPBPhXnDg&WvgBxBHB&ngE--S&qwDSiVPPBSSPnRg-LgPghLXPnBBBBXBXXBHh-ShghBX-PgP-&SRvgBgRRRvvSvSRqvgg&RRSlPhP-PRR-PPB;RRP--RnBBRRR-R:-!;n:DRuBBRvBLGRnBhRGlDnvLln:hLB::RRnvDR,;:EwLnR,RD!gLRnnnhRlRFRlDPPg;WSP:gSv:gg:vS,wll:wPg,,,TlTlLEggDgG,gwL!gGSRwGgw;cg;cvTjGRGRccLLcSGTw;TcB-RnwRBGERcSGSw-TlnTqF-LGSSGw-Wwl&lL-FwPFXRSPH&RF!

In [ ]:
# well that was fun